## Many-to-Many Sequence Problems
In one-to-many and many-to-one sequence problems, we saw that the output vector can contain multiple values. Depending upon the problem, an output vector containing multiple values can be considered as having single (since the output contains one time-step data in strict terms) or multiple (since one vector contains multiple values) outputs.

However, in some sequence problems, we want multiple outputs divided over time-steps. In other words, for each time-step in the input, we want a corresponding time-step in the output. Such models can be used to solve many-to-many sequence problems with variable lengths.

## Encoder-Decoder Model
To solve such sequence problems, the encoder-decoder model has been designed. The encoder-decoder model is basically a fancy name for neural network architecture with two LSTM layers.

The first layer works as an encoder layer and encodes the input sequence. The decoder is also an LSTM layer, which accepts three inputs: the encoded sequence from the encoder LSTM, the previous hidden state, and the current input. During training the actual output at each time-step is used to train the encoder-decoder model. While making predictions, the encoder output, the current hidden state, and the previous output is used as input to make prediction at each time-step. These concepts will become more understandable when you will see them in action in an upcoming section.

## Many-to-Many Sequence Problems with Single Feature
In this section we will solve many-to-many sequence problems via the encoder-decoder model, where each time-step in the input sample will contain one feature.

Let's first create our dataset.

## Creating the Dataset

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

The input X contains 20 samples where each sample contains 3 time-steps with one feature. One input sample looks like this: </br>
[[[  5]  [ 10]  [ 15]]
</br>
You can see that the input sample contain 3 values that are basically 3 consecutive multiples of 5. The corresponding output sequence for the above input sample is as follows:</br>
[[[ 20]  [ 25]  [ 30]]</br>


The output contains the next three consecutive multiples of 5. You can see the output in this case is different than what we have seen in the previous sections. For the encoder-decoder model, the output should also be converted into a 3D format containing the number of samples, time-steps, and features. The is because the decoder generates an output per time-step

We have created our dataset; the next step is to train our models. We will train stacked LSTM and bidirectional LSTM models in the following sections.

## Solution via Stacked LSTM
The following script creates the encoder-decoder model using stacked LSTMs:

In [3]:
from tensorflow.keras.layers import RepeatVector, TimeDistributed

model = Sequential()

# Encoder layer
model.add(LSTM(100, activation="relu", input_shape=(3,1)))

# Repeat vector
model.add(RepeatVector(3))

# Decoder Layer
model.add(LSTM(100, activation="relu", return_sequences=True))

model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               40800     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 3, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 100)            80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 1)              101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
None


In the above script, the first LSTM layer is the encoder layer.

Next, we have added the repeat vector to our model. The repeat vector takes the output from encoder and feeds it repeatedly as input at each time-step to the decoder. For instance, in the output we have three time-steps. To predict each output time-step, the decoder will use the value from the repeat vector, the hidden state from the previous output and the current input.

Next we have a decoder layer. Since the output is in the form of a time-step, which is a 3D format, the return_sequences for the decoder model has been set True. The TimeDistributed layer is used to individually predict the output for each time-step.

The model summary for the encoder-decoder model created in the script above is as follows:

<b>You can see that the repeat vector only repeats the encoder output and has no parameters to train.

The following script trains the above encoder-decoder model.</b>

In [4]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Epoch 1/1000
6/6 [==============================] - 0s 45ms/step - loss: 21909.6582 - val_loss: 68032.7891
Epoch 2/1000
6/6 [==============================] - 0s 7ms/step - loss: 18317.9707 - val_loss: 47108.4805
Epoch 3/1000
6/6 [==============================] - 0s 7ms/step - loss: 11821.9883 - val_loss: 20951.2422
Epoch 4/1000
6/6 [==============================] - 0s 7ms/step - loss: 5644.8232 - val_loss: 17124.7520
Epoch 5/1000
6/6 [==============================] - 0s 8ms/step - loss: 4374.6152 - val_loss: 13128.3047
Epoch 6/1000
6/6 [==============================] - 0s 8ms/step - loss: 3296.9524 - val_loss: 11247.2119
Epoch 7/1000
6/6 [==============================] - 0s 8ms/step - loss: 2663.7131 - val_loss: 8159.7646
Epoch 8/1000
6/6 [==============================] - 0s 7ms/step - loss: 1852.0691 - val_loss: 5122.7222
Epoch 9/1000
6/6 [==============================] - 0s 8ms/step - loss: 1122.7120 - val_loss: 3012.6602
Epoch 10/1000
6/6 [==============================] - 0

Let's create a test-point and see if our encoder-decoder model is able to predict the multi-step output. Execute the following script:

In [5]:
test_input = array([300, 305, 310])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(f"Actual: [315, 320, 325]  ||  Predicted: {test_output}")

Actual: [315, 320, 325]  ||  Predicted: [[[315.72153]
  [321.30884]
  [326.67633]]]


Our input sequence contains three time-step values 300, 305 and 310. The output should be next three multiples of 5 i.e. 315, 320 and 325. I received the following output: 
<b>You can see that the output is in 3D format.</b>

## Solution via Bidirectional LSTM
Let's now create encoder-decoder model with bidirectional LSTMs and see if we can get better results:

In [4]:
from tensorflow.keras.layers import Bidirectional
import tensorflow as tf

model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu', input_shape=(3,1))))
model.add(RepeatVector(3))
model.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

X = tf.cast(X, tf.float32)
Y = tf.cast(Y, tf.float32)
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Epoch 1/1000
6/6 [==============================] - 1s 122ms/step - loss: 20396.9141 - val_loss: 48636.6758
Epoch 2/1000
6/6 [==============================] - 0s 19ms/step - loss: 7533.0630 - val_loss: 2159.7588
Epoch 3/1000
6/6 [==============================] - 0s 19ms/step - loss: 1734.6842 - val_loss: 8192.3008
Epoch 4/1000
6/6 [==============================] - 0s 19ms/step - loss: 716.7704 - val_loss: 780.4224
Epoch 5/1000
6/6 [==============================] - 0s 18ms/step - loss: 619.1168 - val_loss: 1626.1343
Epoch 6/1000
6/6 [==============================] - 0s 20ms/step - loss: 350.8840 - val_loss: 557.4407
Epoch 7/1000
6/6 [==============================] - 0s 19ms/step - loss: 126.9929 - val_loss: 212.4787
Epoch 8/1000
6/6 [==============================] - 0s 18ms/step - loss: 37.5504 - val_loss: 69.6755
Epoch 9/1000
6/6 [==============================] - 0s 19ms/step - loss: 29.0700 - val_loss: 17.1420
Epoch 10/1000
6/6 [==============================] - 0s 19ms/step -

The above script trains the encoder-decoder model via bidirectional LSTM. Let's now make predictions on the test point i.e. [300, 305, 310].

In [6]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: [315, 320, 325]  ||  Predicted: {test_output}")

Actual: [315, 320, 325]  ||  Predicted: [[[315.72153]
  [321.30884]
  [326.67633]]]


## Many-to-Many Sequence Problems with Multiple Features
As you might have guessed it by now, in many-to-many sequence problems, each time-step in the input sample contains multiple features.

Creating the Dataset
Let's create a simple dataset for our problem:

In [2]:
X = list()
Y = list()
X1 = [x1 for x1 in range(5, 301, 5)]
X2 = [x2 for x2 in range(20, 316, 5)]
Y = [y for y in range(35, 331, 5)]

X = np.column_stack((X1, X2))

In the script above we create two lists X1 and X2. The list X1 contains all the multiples of 5 from 5 to 300 (inclusive) and the list X2 contains all the multiples of 5 from 20 to 315 (inclusive). Finally, the list Y, which happens to be the output contains all the multiples of 5 between 35 and 330 (inclusive). The final input list X is a column-wise merger of X1 and X2.

As always, we need to reshape our input X and output Y before they can be used to train LSTM.

In [3]:
X = np.array(X).reshape(20, 3, 2)
Y = np.array(Y).reshape(20, 3, 1)

You can see the input X has been reshaped into 20 samples of three time-steps with 2 features where the output has been reshaped into similar dimensions but with 1 feature.

The input contains 6 consecutive multiples of integer 5, three each in the two columns. Here is the corresponding output for the above input sample:

Let's now train our encoder-decoder model to learn the above sequence. We will first train a simple stacked LSTM-based encoder-decoder.

## Solution via Stacked LSTM
The following script trains the stacked LSTM model. You can see that the input shape is now (3, 2) corresponding to three time-steps and two features in the input.

In [6]:
from tensorflow.keras.layers import RepeatVector, TimeDistributed

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(3,2)))
model.add(RepeatVector(3))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Let's now create a test point that will be used for making a prediction.

In [4]:
X1 = [300, 305, 310]
X2 = [315, 320, 325]

test_input = np.column_stack((X1, X2))

test_input = test_input.reshape((1, 3, 2))
test_output = model.predict(test_input, verbose=0)
print(f"Actual: [330, 335, 340] || Pre: {test_output}")

NameError: name 'model' is not defined

## Solution via Bidirectional LSTM
Let's now train encoder-decoder model based on bidirectional LSTMs and see if we can get improved results. The following script trains the model.

In [9]:
import tensorflow as tf

model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu', input_shape=(3, 2))))
model.add(RepeatVector(3))
model.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

X = tf.cast(X, tf.float32)
Y = tf.cast(Y, tf.float32)

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Epoch 1/1000
6/6 [==============================] - 1s 128ms/step - loss: 20690.8574 - val_loss: 38189.0938
Epoch 2/1000
6/6 [==============================] - 0s 18ms/step - loss: 6886.5649 - val_loss: 233.5182
Epoch 3/1000
6/6 [==============================] - 0s 19ms/step - loss: 1088.5569 - val_loss: 2890.6665
Epoch 4/1000
6/6 [==============================] - 0s 19ms/step - loss: 240.7317 - val_loss: 52.5697
Epoch 5/1000
6/6 [==============================] - 0s 18ms/step - loss: 127.4609 - val_loss: 546.7745
Epoch 6/1000
6/6 [==============================] - 0s 18ms/step - loss: 61.6004 - val_loss: 110.1081
Epoch 7/1000
6/6 [==============================] - 0s 19ms/step - loss: 22.8596 - val_loss: 52.9571
Epoch 8/1000
6/6 [==============================] - 0s 19ms/step - loss: 15.2751 - val_loss: 58.6709
Epoch 9/1000
6/6 [==============================] - 0s 19ms/step - loss: 10.2494 - val_loss: 42.5612
Epoch 10/1000
6/6 [==============================] - 0s 20ms/step - loss:

In [10]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: [330, 335, 340] || Pre: {test_output}")

Actual: [330, 335, 340] || Pre: [[[331.52658]
  [336.73236]
  [342.9897 ]]]


The output achieved is pretty close to the actual output i.e. [330, 335, 340]. Hence our bidirectional LSTM outperformed the simple LSTM.

## Conclusion
This is the second part of my article on "Solving Sequence Problems with LSTM in Keras" (part 1 here). In this article you saw how to solve one-to-many and many-to-many sequence problems in LSTM. You also saw how encoder-decoder model can be used to predict multi-step outputs. The encoder-decoder model is used in a variety of natural language processing applications such as neural machine translation and chatbot development.